## Random forests for predicting power output

In this notebook we will do a random forest regression on our dataset

In [46]:
import pandas as pd
import copy
import numpy as np

In [47]:
ogdf = pd.read_csv('./data/final_dataframes/main_training_dataframe.csv')

In [48]:
ogdf

,Unnamed: 0,time,temperature_2m_1,relative_humidity_2m_1,wind_speed_10m_1,wind_direction_10m_1,temperature_2m_33,relative_humidity_2m_33,wind_speed_10m_33,wind_direction_10m_33,...,relative_humidity_2m_34,wind_speed_10m_34,wind_direction_10m_34,temperature_2m_38,relative_humidity_2m_38,wind_speed_10m_38,wind_direction_10m_38,Wind,Year,MonthDay
0,0,2019-01-01 00:00:00,-7.4,81,17.9,146,-5.7,87,12.4,100,...,88,13.0,132,-5.6,84,18.8,144,1933.0,2019,01-01
1,1,2019-01-01 01:00:00,-7.0,80,18.9,145,-5.8,87,13.6,102,...,87,14.6,130,-5.8,84,19.2,142,1996.0,2019,01-01
2,2,2019-01-01 02:00:00,-7.1,80,21.3,146,-5.9,88,12.9,103,...,89,16.0,121,-5.8,83,17.9,142,1936.0,2019,01-01
3,3,2019-01-01 03:00:00,-6.8,80,22.2,148,-5.9,90,15.9,103,...,87,16.2,122,-6.0,84,15.0,136,2023.0,2019,01-01
4,4,2019-01-01 04:00:00,-6.8,81,22.4,143,-5.9,90,15.6,109,...,86,16.1,114,-6.3,86,15.8,131,2147.0,2019,01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,35059,2022-12-31 19:00:00,3.1,92,7.7,217,1.1,94,5.6,45,...,97,4.9,234,1.8,98,4.6,198,145.0,2022,12-31
35060,35060,2022-12-31 20:00:00,2.6,97,2.5,188,1.0,95,4.6,51,...,98,1.1,90,2.2,100,5.1,188,118.0,2022,12-31
35061,35061,2022-12-31 21:00:00,2.9,96,2.7,203,0.8,96,6.0,17,...,99,1.8,90,3.4,99,2.2,189,31.0,2022,12-31
35062,35062,2022-12-31 22:00:00,3.8,94,3.1,54,0.6,96,5.2,335,...,99,3.3,96,3.0,100,1.8,127,33.0,2022,12-31


In [49]:
#I am now going to clean up some columns and add features
houroutdf=copy.deepcopy(ogdf)
houroutdf = houroutdf.drop(columns= ["Unnamed: 0"])
houroutdf['Month'] = houroutdf['MonthDay'].str[:2]
houroutdf['time_of_day'] = houroutdf['time'].str[10:]
# Convert to datetime and extract hour
houroutdf['time_of_day'] = houroutdf['time_of_day'].str[:3].astype(int)
houroutdf = houroutdf.drop(columns=['time'])
houroutdf = houroutdf[[col for col in houroutdf.columns if col != "Wind"] + ["Wind"]]
houroutdf = houroutdf.rename(columns={'Wind': 'Wind_Power'})
df0 = copy.deepcopy(houroutdf) #removing categorical variables for ease rn
df0 = df0.drop(columns=['MonthDay'])
df0

,temperature_2m_1,relative_humidity_2m_1,wind_speed_10m_1,wind_direction_10m_1,temperature_2m_33,relative_humidity_2m_33,wind_speed_10m_33,wind_direction_10m_33,temperature_2m_2,relative_humidity_2m_2,...,wind_speed_10m_34,wind_direction_10m_34,temperature_2m_38,relative_humidity_2m_38,wind_speed_10m_38,wind_direction_10m_38,Year,Month,time_of_day,Wind_Power
0,-7.4,81,17.9,146,-5.7,87,12.4,100,-13.4,85,...,13.0,132,-5.6,84,18.8,144,2019,01,0,1933.0
1,-7.0,80,18.9,145,-5.8,87,13.6,102,-13.6,87,...,14.6,130,-5.8,84,19.2,142,2019,01,1,1996.0
2,-7.1,80,21.3,146,-5.9,88,12.9,103,-14.1,89,...,16.0,121,-5.8,83,17.9,142,2019,01,2,1936.0
3,-6.8,80,22.2,148,-5.9,90,15.9,103,-12.8,86,...,16.2,122,-6.0,84,15.0,136,2019,01,3,2023.0
4,-6.8,81,22.4,143,-5.9,90,15.6,109,-11.9,88,...,16.1,114,-6.3,86,15.8,131,2019,01,4,2147.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,3.1,92,7.7,217,1.1,94,5.6,45,-2.5,99,...,4.9,234,1.8,98,4.6,198,2022,12,19,145.0
35060,2.6,97,2.5,188,1.0,95,4.6,51,-1.1,100,...,1.1,90,2.2,100,5.1,188,2022,12,20,118.0
35061,2.9,96,2.7,203,0.8,96,6.0,17,-0.9,99,...,1.8,90,3.4,99,2.2,189,2022,12,21,31.0
35062,3.8,94,3.1,54,0.6,96,5.2,335,3.0,90,...,3.3,96,3.0,100,1.8,127,2022,12,22,33.0


In [62]:
#now we need to take the entries in 'Wind_power' and then add n=24 of them together for power one day ahead
n=24
dayoutdf = copy.deepcopy(houroutdf)
dayoutdf['Wind_Power'] = dayoutdf['Wind_Power'].rolling(window=n).sum().shift(-n) #this replaces 'Wind' time t with the sum of the entries in 'Wind' at t+1,... t+24
df = copy.deepcopy(dayoutdf) #removing categorical variables for ease rn
df = df.drop(columns=['MonthDay'])
df


,temperature_2m_1,relative_humidity_2m_1,wind_speed_10m_1,wind_direction_10m_1,temperature_2m_33,relative_humidity_2m_33,wind_speed_10m_33,wind_direction_10m_33,temperature_2m_2,relative_humidity_2m_2,...,wind_speed_10m_34,wind_direction_10m_34,temperature_2m_38,relative_humidity_2m_38,wind_speed_10m_38,wind_direction_10m_38,Year,Month,time_of_day,Wind_Power
0,-7.4,81,17.9,146,-5.7,87,12.4,100,-13.4,85,...,13.0,132,-5.6,84,18.8,144,2019,01,0,53069.0
1,-7.0,80,18.9,145,-5.8,87,13.6,102,-13.6,87,...,14.6,130,-5.8,84,19.2,142,2019,01,1,53355.0
2,-7.1,80,21.3,146,-5.9,88,12.9,103,-14.1,89,...,16.0,121,-5.8,83,17.9,142,2019,01,2,53663.0
3,-6.8,80,22.2,148,-5.9,90,15.9,103,-12.8,86,...,16.2,122,-6.0,84,15.0,136,2019,01,3,53689.0
4,-6.8,81,22.4,143,-5.9,90,15.6,109,-11.9,88,...,16.1,114,-6.3,86,15.8,131,2019,01,4,53600.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,3.1,92,7.7,217,1.1,94,5.6,45,-2.5,99,...,4.9,234,1.8,98,4.6,198,2022,12,19,NaN
35060,2.6,97,2.5,188,1.0,95,4.6,51,-1.1,100,...,1.1,90,2.2,100,5.1,188,2022,12,20,NaN
35061,2.9,96,2.7,203,0.8,96,6.0,17,-0.9,99,...,1.8,90,3.4,99,2.2,189,2022,12,21,NaN
35062,3.8,94,3.1,54,0.6,96,5.2,335,3.0,90,...,3.3,96,3.0,100,1.8,127,2022,12,22,NaN


In [6]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_absolute_percentage_error, mean_absolute_error, mean_squared_error, r2_score, make_scorer

In [15]:
a = np.random.randint(low= 0, high=27000, size=None, dtype=int)
b = np.random.randint(low= 0, high=6000, size=None, dtype=int)

In [36]:
start_time =  np.random.randint(low= 0, high=27000, size=None, dtype=int) #start time
end_time = start_time + np.random.randint(low= 0, high=6000, size=None, dtype=int) #end time
testing_interval = 240 #amount of time in the future to predict
print(start_time)
print(end_time)

2914
7709


In [37]:

traindf = df.iloc[start_time:end_time]   # first 3 years
testdf = df.iloc[end_time:(end_time+testing_interval)] # next day

X_train = traindf[[col for col in df.columns if col != "Wind_Power"]]
y_train = traindf["Wind_Power"]

X_test = testdf[[col for col in df.columns if col != "Wind_Power"]]
y_test = testdf["Wind_Power"]



In [38]:
#dummy regressor
dummy = DummyRegressor(strategy='mean')  # You can also use strategy='constant', constant=YOUR_VALUE
dummy.fit(X_train, y_train)

# Predictions
y_pred = dummy.predict(X_test)

# Evaluation
print("MAPE:", mean_absolute_percentage_error(y_test, y_pred))
print("R^2 Score:", r2_score(y_test, y_pred))


MAPE: 1.3104756476056418
R^2 Score: -0.011080195625124967


In [39]:
param_grid = {
    'n_estimators': [5, 10, 20, 50, 100, 200],
    'max_depth': [5, 10, 20, 50, 100, 200],
    'bootstrap': [True, False]
}
scoring = {
    'MAPE': make_scorer(mean_absolute_percentage_error, greater_is_better=False),
    'R^2 Score': make_scorer(r2_score, greater_is_better=False)
}
rf = RandomForestRegressor(random_state=614)

In [40]:
results = []

for n_estimators in param_grid['n_estimators']:
    for max_depth in param_grid['max_depth']:
        for bootstrap in param_grid['bootstrap']:
            model = RandomForestRegressor(
                n_estimators=n_estimators,
                max_depth=max_depth,
                bootstrap=bootstrap,
                random_state=614
            )
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)

            mape = mean_absolute_percentage_error(y_test, y_pred)
            r2 = r2_score(y_test, y_pred)

            results.append({
                'n_estimators': n_estimators,
                'max_depth': max_depth,
                'bootstrap': bootstrap,
                'MAPE (%)': mape,
                'R2 Score': r2
            })

results_df = pd.DataFrame(results)
print(results_df.sort_values(by='MAPE (%)'))

    n_estimators  max_depth  bootstrap  MAPE (%)  R2 Score
4              5         20       True  0.741483  0.534819
0              5          5       True  0.756429  0.567521
63           200         10      False  0.763111  0.428408
51           100         10      False  0.764509  0.427909
15            10         10      False  0.769236  0.402632
..           ...        ...        ...       ...       ...
58           100        200       True  0.857944  0.532377
56           100        100       True  0.857944  0.532377
66           200         50       True  0.865795  0.551076
68           200        100       True  0.865795  0.551076
70           200        200       True  0.865795  0.551076

[72 rows x 5 columns]


In [42]:
results_df.to_excel("2914_to_7709_240hrs_results_output.xlsx", index=False)

# Let's try only using wind speed

In [90]:
#making df with only windspeed and wind power
windspeed_col = []
for i in range(0,40):
    windspeed_col.append(f"wind_speed_10m_{i}")  # appends i squared
windspeed_col.append('Wind_Power')
wsdf = df[[col for col in windspeed_col if col in df.columns]]
wsdf

,wind_speed_10m_1,wind_speed_10m_2,wind_speed_10m_3,wind_speed_10m_4,wind_speed_10m_5,wind_speed_10m_6,wind_speed_10m_7,wind_speed_10m_8,wind_speed_10m_9,wind_speed_10m_10,...,wind_speed_10m_31,wind_speed_10m_32,wind_speed_10m_33,wind_speed_10m_34,wind_speed_10m_35,wind_speed_10m_36,wind_speed_10m_37,wind_speed_10m_38,wind_speed_10m_39,Wind_Power
0,17.9,11.4,14.5,18.0,14.3,12.2,11.9,11.7,12.8,13.4,...,14.0,9.9,12.4,13.0,12.7,15.3,18.2,18.8,13.9,53069.0
1,18.9,12.8,14.8,19.4,15.5,13.1,12.9,8.1,14.3,14.3,...,13.8,9.7,13.6,14.6,14.5,16.3,18.7,19.2,10.8,53355.0
2,21.3,13.6,14.2,18.9,16.1,13.4,14.6,6.8,14.5,14.9,...,14.0,9.2,12.9,16.0,14.5,17.4,19.9,17.9,12.3,53663.0
3,22.2,15.5,15.1,19.6,17.3,13.1,15.0,7.2,14.8,14.8,...,14.1,8.9,15.9,16.2,15.1,16.7,20.9,15.0,10.0,53689.0
4,22.4,16.2,16.9,15.9,16.6,15.0,16.7,9.7,15.3,15.3,...,15.3,10.8,15.6,16.1,14.4,16.9,17.3,15.8,13.0,53600.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,7.7,5.8,8.7,6.5,4.9,6.4,5.3,4.3,4.6,4.4,...,4.0,3.0,5.6,4.9,5.4,3.8,6.2,4.6,22.4,NaN
35060,2.5,3.2,5.9,6.6,6.8,6.6,9.7,2.8,2.6,3.9,...,5.5,4.0,4.6,1.1,3.3,4.9,7.9,5.1,18.6,NaN
35061,2.7,0.8,4.0,6.6,7.2,3.6,7.4,1.5,2.9,3.6,...,4.0,2.5,6.0,1.8,3.2,4.0,6.1,2.2,7.8,NaN
35062,3.1,2.8,4.7,6.2,6.0,5.0,6.0,2.6,2.9,3.1,...,0.5,3.6,5.2,3.3,1.9,3.3,5.9,1.8,5.6,NaN


In [91]:
#dummy regressor
dummy = DummyRegressor(strategy='mean')  # You can also use strategy='constant', constant=YOUR_VALUE
dummy.fit(X_train, y_train)

# Predictions
y_pred = dummy.predict(X_test)

# Evaluation
print("MAPE:", mean_absolute_percentage_error(y_test, y_pred))
print("R^2 Score:", r2_score(y_test, y_pred))


MAPE: 2.781807420582506
R^2 Score: -0.011080195625124967


In [ ]:
results = []

for n_estimators in param_grid['n_estimators']:
    for max_depth in param_grid['max_depth']:
        for bootstrap in param_grid['bootstrap']:
            model = RandomForestRegressor(
                n_estimators=n_estimators,
                max_depth=max_depth,
                bootstrap=bootstrap,
                random_state=614
            )
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)

            mape = mean_absolute_percentage_error(y_test, y_pred)
            r2 = r2_score(y_test, y_pred)

            results.append({
                'n_estimators': n_estimators,
                'max_depth': max_depth,
                'bootstrap': bootstrap,
                'MAPE (%)': mape,
                'R2 Score': r2
            })

results_df = pd.DataFrame(results)
print(results_df.sort_values(by='MAPE (%)'))
results_df.to_excel("Wind_speed_only_2914_to_7709_240hrs_results_output.xlsx", index=False)